In [1]:
import pandas as pd
import ast
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import numpy as np
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# get Indonesian stopword 
list_stopwords = set(stopwords.words('indonesian'))

# factory = StopWordRemoverFactory()
# stopword = factory.create_stop_word_remover()
# fact = StemmerFactory()
# stemmer = fact.create_stemmer()

In [2]:
df = pd.read_csv('./dataset/database_konten_fixed.csv')
df.head()

,id,title,summary,imgUrl,content,author,createdAt,tags,link,viewers,likes
0,61a420e4-0044-11ee-bbbb-bb155eb672e5,Tips Menjaga Kebugaran Lansia yang Perlu Diket...,"“Seiring dengan bertambahnya umur, menjaga keb...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seiring bertambahnya usia, ...",dr. Rizal Fadli,29 Mei 2023,"[""kesehatan"",""lansia"",""kebugaran""]",https://www.halodoc.com/artikel/tips-menjaga-k...,0,0
1,61b5264f-0044-11ee-ac36-bb155eb672e5,"Jarang Diketahui, Ini 12 Manfaat Daun Seledri ...",“Ada berbagai manfaat seledri untuk kesehatan....,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seledri merupakan sayuran y...",dr. Fadhli Rizal Makarim,30 Mei 2023,"[""kesehatan"",""makanan""]",https://www.halodoc.com/artikel/jarang-diketah...,0,0
2,61c727ae-0044-11ee-bd96-bb155eb672e5,9 Obat Herbal yang Ampuh Atasi Sakit Lutut dan...,“Obat herbal dapat menjadi pilihan alternatif ...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Sakit lutut dan sendi adala...",dr. Rizal Fadli,10 Mei 2023,"[""herbal"",""kesehatan"",""obat""]",https://www.halodoc.com/artikel/9-obat-herbal-...,0,0
3,61dae8d1-0044-11ee-ac0f-bb155eb672e5,5 Gerakan Yoga yang Bisa Dilakukan oleh Pemula,NaN,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,“Yoga adalah olahraga yang terbilang cukup rin...,dr. Rizal Fadli,03 Desember 2021,"[""olahraga"",""yoga"",""pemula""]",https://www.halodoc.com/artikel/5-gerakan-yoga...,0,0
4,61eb98ae-0044-11ee-a45a-bb155eb672e5,Lakukan 5 Gerakan Yoga Ini untuk Meredakan Stres,NaN,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Yoga dapat membantu mengura...",dr. Verury Verona Handayani,18 Desember 2020,"[""yoga"",""olahraga"",""kesehatan mental"",""stres""]",https://www.halodoc.com/artikel/lakukan-5-gera...,0,0


In [24]:
# df.to_json('database_konten_fixed2.json',orient='records')

In [25]:
# pd.read_json('database_konten_fixed2.json')

,id,title,summary,imgUrl,content,author,createdAt,tags,link,viewers,likes,text,cosim
0,61a420e4-0044-11ee-bbbb-bb155eb672e5,Tips Menjaga Kebugaran Lansia yang Perlu Diket...,"“Seiring dengan bertambahnya umur, menjaga keb...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seiring bertambahnya usia, ...",dr. Rizal Fadli,29 Mei 2023,kesehatan lansia kebugaran,https://www.halodoc.com/artikel/tips-menjaga-k...,0,0,tips menjaga kebugaran lansia seiring bertamba...,0.112040
1,61b5264f-0044-11ee-ac36-bb155eb672e5,"Jarang Diketahui, Ini 12 Manfaat Daun Seledri ...",“Ada berbagai manfaat seledri untuk kesehatan....,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seledri merupakan sayuran y...",dr. Fadhli Rizal Makarim,30 Mei 2023,kesehatan makanan,https://www.halodoc.com/artikel/jarang-diketah...,0,0,jarang manfaat daun seledri kesehatan manfaat...,0.014120
2,61c727ae-0044-11ee-bd96-bb155eb672e5,9 Obat Herbal yang Ampuh Atasi Sakit Lutut dan...,“Obat herbal dapat menjadi pilihan alternatif ...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Sakit lutut dan sendi adala...",dr. Rizal Fadli,10 Mei 2023,herbal kesehatan obat,https://www.halodoc.com/artikel/9-obat-herbal-...,0,0,obat herbal ampuh atasi sakit lutut sendi obat...,0.000000
3,61dae8d1-0044-11ee-ac0f-bb155eb672e5,5 Gerakan Yoga yang Bisa Dilakukan oleh Pemula,,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,“Yoga adalah olahraga yang terbilang cukup rin...,dr. Rizal Fadli,03 Desember 2021,olahraga yoga pemula,https://www.halodoc.com/artikel/5-gerakan-yoga...,0,0,gerakan yoga pemula yoga olahraga terbilang ...,0.000000
4,61eb98ae-0044-11ee-a45a-bb155eb672e5,Lakukan 5 Gerakan Yoga Ini untuk Meredakan Stres,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Yoga dapat membantu mengura...",dr. Verury Verona Handayani,18 Desember 2020,yoga olahraga kesehatan mental stres,https://www.halodoc.com/artikel/lakukan-5-gera...,0,0,lakukan gerakan yoga meredakan stres halodo...,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,6f70242b-0044-11ee-8828-bb155eb672e5,5 Gangguan Kepribadian dengan Rasa Cemas Berle...,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Gangguan kepribadian adalah...",Redaksi Halodoc,29 Oktober 2018,cemas psikologi,https://www.halodoc.com/artikel/5-gangguan-kep...,0,0,gangguan kepribadian cemas halodoc jakarta ...,0.022351
186,6f8184a4-0044-11ee-9f3f-bb155eb672e5,15 Gejala yang Timbul dari Gangguan Kecemasan,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Siapa yang tidak mengenal g...",dr. Verury Verona Handayani,18 Februari 2021,cemas psikologi kesehatan mental,https://www.halodoc.com/artikel/15-gejala-yang...,0,0,gejala timbul gangguan kecemasan halodoc jak...,0.014028
187,6f94709e-0044-11ee-a563-bb155eb672e5,Latihan Pernapasan yang Bisa Meredakan Kecemasan,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Meskipun merupakan perasaan...",dr. Fadhli Rizal Makarim,21 Januari 2021,cemas olahraga,https://www.halodoc.com/artikel/latihan-pernap...,0,0,latihan pernapasan meredakan kecemasan halod...,0.000000
188,6fac42af-0044-11ee-ae14-bb155eb672e5,"Makanan Plant Based Bisa Meredakan Kecemasan, ...",,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Kesehatan mental sama penti...",dr. Verury Verona Handayani,27 November 2020,cemas makanan,https://www.halodoc.com/artikel/makanan-plant-...,0,0,makanan plant based meredakan kecemasan faktan...,0.028703


In [4]:
df['tags'] = df['tags'].apply(lambda x: " ".join(ast.literal_eval(x)))
df['tags']

0                kesehatan lansia kebugaran
1                         kesehatan makanan
2                     herbal kesehatan obat
3                      olahraga yoga pemula
4      yoga olahraga kesehatan mental stres
                       ...                 
185                         cemas psikologi
186        cemas psikologi kesehatan mental
187                          cemas olahraga
188                           cemas makanan
189                           cemas makanan
Name: tags, Length: 190, dtype: object

In [5]:
same_summary = []
for i in range(df.shape[0]):
    if df.loc[i,'summary'] not in same_summary:
        same_summary.append(df.loc[i,'summary'])
    else:
        df.loc[i,'summary'] = ' '

In [6]:
# df.to_csv('dataset_konten_fixed.csv',index=False)
# df.to_json('database_konten_fixed.json',orient='records')

In [7]:
df.fillna(' ',inplace=True)

In [8]:
df['text'] = df['title']+' '+df['summary']+' '+df['content'] + ' ' + df['tags']
# df.drop(['title','summary','content','tags','imgUrl','link'],axis=1,inplace=True)

In [9]:
df

,id,title,summary,imgUrl,content,author,createdAt,tags,link,viewers,likes,text
0,61a420e4-0044-11ee-bbbb-bb155eb672e5,Tips Menjaga Kebugaran Lansia yang Perlu Diket...,"“Seiring dengan bertambahnya umur, menjaga keb...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seiring bertambahnya usia, ...",dr. Rizal Fadli,29 Mei 2023,kesehatan lansia kebugaran,https://www.halodoc.com/artikel/tips-menjaga-k...,0,0,Tips Menjaga Kebugaran Lansia yang Perlu Diket...
1,61b5264f-0044-11ee-ac36-bb155eb672e5,"Jarang Diketahui, Ini 12 Manfaat Daun Seledri ...",“Ada berbagai manfaat seledri untuk kesehatan....,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seledri merupakan sayuran y...",dr. Fadhli Rizal Makarim,30 Mei 2023,kesehatan makanan,https://www.halodoc.com/artikel/jarang-diketah...,0,0,"Jarang Diketahui, Ini 12 Manfaat Daun Seledri ..."
2,61c727ae-0044-11ee-bd96-bb155eb672e5,9 Obat Herbal yang Ampuh Atasi Sakit Lutut dan...,“Obat herbal dapat menjadi pilihan alternatif ...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Sakit lutut dan sendi adala...",dr. Rizal Fadli,10 Mei 2023,herbal kesehatan obat,https://www.halodoc.com/artikel/9-obat-herbal-...,0,0,9 Obat Herbal yang Ampuh Atasi Sakit Lutut dan...
3,61dae8d1-0044-11ee-ac0f-bb155eb672e5,5 Gerakan Yoga yang Bisa Dilakukan oleh Pemula,,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,“Yoga adalah olahraga yang terbilang cukup rin...,dr. Rizal Fadli,03 Desember 2021,olahraga yoga pemula,https://www.halodoc.com/artikel/5-gerakan-yoga...,0,0,5 Gerakan Yoga yang Bisa Dilakukan oleh Pemula...
4,61eb98ae-0044-11ee-a45a-bb155eb672e5,Lakukan 5 Gerakan Yoga Ini untuk Meredakan Stres,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Yoga dapat membantu mengura...",dr. Verury Verona Handayani,18 Desember 2020,yoga olahraga kesehatan mental stres,https://www.halodoc.com/artikel/lakukan-5-gera...,0,0,Lakukan 5 Gerakan Yoga Ini untuk Meredakan Str...
...,...,...,...,...,...,...,...,...,...,...,...,...
185,6f70242b-0044-11ee-8828-bb155eb672e5,5 Gangguan Kepribadian dengan Rasa Cemas Berle...,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Gangguan kepribadian adalah...",Redaksi Halodoc,29 Oktober 2018,cemas psikologi,https://www.halodoc.com/artikel/5-gangguan-kep...,0,0,5 Gangguan Kepribadian dengan Rasa Cemas Berle...
186,6f8184a4-0044-11ee-9f3f-bb155eb672e5,15 Gejala yang Timbul dari Gangguan Kecemasan,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Siapa yang tidak mengenal g...",dr. Verury Verona Handayani,18 Februari 2021,cemas psikologi kesehatan mental,https://www.halodoc.com/artikel/15-gejala-yang...,0,0,15 Gejala yang Timbul dari Gangguan Kecemasan ...
187,6f94709e-0044-11ee-a563-bb155eb672e5,Latihan Pernapasan yang Bisa Meredakan Kecemasan,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Meskipun merupakan perasaan...",dr. Fadhli Rizal Makarim,21 Januari 2021,cemas olahraga,https://www.halodoc.com/artikel/latihan-pernap...,0,0,Latihan Pernapasan yang Bisa Meredakan Kecemas...
188,6fac42af-0044-11ee-ae14-bb155eb672e5,"Makanan Plant Based Bisa Meredakan Kecemasan, ...",,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Kesehatan mental sama penti...",dr. Verury Verona Handayani,27 November 2020,cemas makanan,https://www.halodoc.com/artikel/makanan-plant-...,0,0,"Makanan Plant Based Bisa Meredakan Kecemasan, ..."


In [10]:
def clean_text(x):
    x = x.lower()
    x = x.strip()
    x = re.sub('[^a-z ]','',x)
    #remove stopword pada list token
    x = ' '.join([word for word in x.split(' ') if not word in list_stopwords])
    # x = stopword.remove(x)
    return x.strip()

In [11]:
df.text

0      Tips Menjaga Kebugaran Lansia yang Perlu Diket...
1      Jarang Diketahui, Ini 12 Manfaat Daun Seledri ...
2      9 Obat Herbal yang Ampuh Atasi Sakit Lutut dan...
3      5 Gerakan Yoga yang Bisa Dilakukan oleh Pemula...
4      Lakukan 5 Gerakan Yoga Ini untuk Meredakan Str...
                             ...                        
185    5 Gangguan Kepribadian dengan Rasa Cemas Berle...
186    15 Gejala yang Timbul dari Gangguan Kecemasan ...
187    Latihan Pernapasan yang Bisa Meredakan Kecemas...
188    Makanan Plant Based Bisa Meredakan Kecemasan, ...
189    Ini Alasan Membuat Kue dapat Redakan Gangguan ...
Name: text, Length: 190, dtype: object

In [12]:
df.text = df.text.apply(clean_text)

In [13]:
df.text

0      tips menjaga kebugaran lansia seiring bertamba...
1      jarang  manfaat daun seledri kesehatan manfaat...
2      obat herbal ampuh atasi sakit lutut sendi obat...
3      gerakan yoga pemula   yoga olahraga terbilang ...
4      lakukan  gerakan yoga meredakan stres   halodo...
                             ...                        
185    gangguan kepribadian cemas   halodoc jakarta  ...
186    gejala timbul gangguan kecemasan   halodoc jak...
187    latihan pernapasan meredakan kecemasan   halod...
188    makanan plant based meredakan kecemasan faktan...
189    alasan kue redakan gangguan kecemasan   halodo...
Name: text, Length: 190, dtype: object

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorizing pre-processed movie plots using TF-IDF
tfidfvec = TfidfVectorizer()
vectors = tfidfvec.fit_transform((df.text)).toarray()
  
user_input = 'hari ini saya ingin berolahraga kalo bisa ingin yoga'
user_input = clean_text(user_input)
vector_user = tfidfvec.transform([user_input]).toarray()

In [15]:
vector_user

array([[0., 0., 0., ..., 0., 0., 0.]])

In [16]:
# Finding cosine similarity between vectors
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(vectors,vector_user)
cos_sim

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.44874588],
       [0.36440062],
       [0.52548598],
       [0.43772322],
       [0.23232681],
       [0.29142877],
       [0.24114642],
       [0.21895982],
       [0.29243951],
       [0.36493331],
       [0.29591775],
       [0.38785474],
       [0.2658656 ],
       [0.51930017],
       [0.26976635],
       [0.39926141],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.09948371],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.01956702],
       [0.02150939],
       [0.02312884],
       [0.01759251],
       [0.        ],
       [0.        ],
       [0.   

In [17]:
import numpy as np
df.iloc[np.argmax(cos_sim)]

id                        62018294-0044-11ee-9667-bb155eb672e5
title        Alasan Yoga Mampu Meredakan Tingkat Stres, Waj...
summary                                                       
imgUrl       https://d1bpj0tv6vfxyp.cloudfront.net/articles...
content      Halodoc, Jakarta – Tekanan pekerjaan atau berb...
author                                         dr. Rizal Fadli
createdAt                                    06 September 2020
tags                      yoga olahraga kesehatan mental stres
link         https://www.halodoc.com/artikel/alasan-yoga-ma...
viewers                                                      0
likes                                                        0
text         alasan yoga meredakan tingkat stres wajib dico...
Name: 5, dtype: object

In [18]:
cos_sim

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.44874588],
       [0.36440062],
       [0.52548598],
       [0.43772322],
       [0.23232681],
       [0.29142877],
       [0.24114642],
       [0.21895982],
       [0.29243951],
       [0.36493331],
       [0.29591775],
       [0.38785474],
       [0.2658656 ],
       [0.51930017],
       [0.26976635],
       [0.39926141],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.09948371],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.01956702],
       [0.02150939],
       [0.02312884],
       [0.01759251],
       [0.        ],
       [0.        ],
       [0.   

In [19]:
df['cosim'] = cos_sim

In [20]:
def recommendations(input_user, cosine_sim = cos_sim):
    user_input = clean_text(input_user)
    vector_user = tfidfvec.transform([user_input]).toarray()
    print(user_input)
    cos_sim = cosine_similarity(vectors,vector_user)
    df['cosim'] = cos_sim
    
    return df.sort_values('cosim',ascending=False).loc[:10]

In [21]:
recommendations(input_user='hari ini saya ingin makan sehat',cosine_sim=cos_sim)

makan sehat


,id,title,summary,imgUrl,content,author,createdAt,tags,link,viewers,likes,text,cosim
118,6a17ed19-0044-11ee-8078-bb155eb672e5,Bagaimana Membedakan Marah yang Sehat dan Tida...,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Kemarahan sering kali diang...",dr. Fadhli Rizal Makarim,26 Januari 2021,marah psikologi,https://www.halodoc.com/artikel/bagaimana-memb...,0,0,membedakan marah sehat sehat halodoc jakarta...,0.236654
34,640067c5-0044-11ee-8b8a-bb155eb672e5,"Catat, Ini 5 Resep Makanan Sehat untuk Ibu Hamil",“Wanita memerlukan nutrisi yang banyak ketika ...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Makanan untuk ibu hamil hen...",dr. Fadhli Rizal Makarim,07 September 2022,makanan makanan sehat kesehatan kehamilan,https://www.halodoc.com/artikel/catat-ini-5-re...,0,0,catat resep makanan sehat hamil wanita nutris...,0.188249
27,638bb166-0044-11ee-8757-bb155eb672e5,Ide Makanan Sehat untuk Makan Malam Keluarga,"""Menu makan malam keluarga juga perlu diperhat...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Salah satu kunci kesehatan ...",dr. Fadhli Rizal Makarim,28 Desember 2022,makanan makanan sehat kesehatan keluarga,https://www.halodoc.com/artikel/ide-makanan-se...,0,0,ide makanan sehat makan malam keluarga menu ma...,0.177393
19,63041159-0044-11ee-8c98-bb155eb672e5,Ini Berbagai Makanan Sehat dan Bergizi untuk L...,“Menjaga kesehatan lansia dimulai dari memberi...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Mengonsumsi makanan sehat t...",dr. Rizal Fadli,29 Mei 2023,makanan makanan sehat lansia,https://www.halodoc.com/artikel/ini-berbagai-m...,0,0,makanan sehat bergizi lansia menjaga kesehatan...,0.176604
23,63474913-0044-11ee-8b72-bb155eb672e5,Kenali Apa Saja Makanan 4 Sehat 5 Sempurna dan...,“Ibu pasti sudah tidak asing dengan menu makan...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Makanan 4 sehat 5 sempurna ...",dr. Rizal Fadli,04 Februari 2023,makanan makanan sehat kesehatan,https://www.halodoc.com/artikel/kenali-apa-saj...,0,0,kenali makanan sehat sempurna manfaatnya asi...,0.171531
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,6d2d93b9-0044-11ee-9608-bb155eb672e5,"Takut Ditinggalkan, Gejala dari Gangguan Kepri...",,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Kenyataannya memang setiap ...",dr. Rizal Fadli,07 Agustus 2020,takut gangguan,https://www.halodoc.com/artikel/takut-ditingga...,0,0,takut ditinggalkan gejala gangguan kepribadian...,0.008984
161,6d95a618-0044-11ee-aff5-bb155eb672e5,"Cemas Bikin Insomnia, Bagaimana Mengatasinya?",,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Rasa cemas merupakan hal no...",dr. Rizal Fadli,26 Maret 2021,cemas takut psikologi kesehatan mental,https://www.halodoc.com/artikel/cemas-bikin-in...,0,0,cemas bikin insomnia mengatasinya halodoc ja...,0.008808
109,697b9205-0044-11ee-8a48-bb155eb672e5,Ketahui Manfaat Bernostalgia bagi Kesehatan Me...,“Ketika seseorang bernostalgia dan berfokus pa...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Apakah kamu termasuk orang...",dr. Rizal Fadli,27 September 2022,kesehatan mental hobi,https://www.halodoc.com/artikel/ketahui-manfaa...,0,0,ketahui manfaat bernostalgia kesehatan mental ...,0.008691
152,6caaa840-0044-11ee-a0eb-bb155eb672e5,Takut Naik Pesawat? Ini 5 Cara Mengatasinya,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Naik pesawat masih menjadi ...",dr. Fadhli Rizal Makarim,11 Januari 2021,takut psikologi fobia,https://www.halodoc.com/artikel/takut-naik-pes...,0,0,takut pesawat mengatasinya halodoc jakarta ...,0.008199


In [23]:
tes = pd.read_json('database_konten2.json',orient='records')
tes

FileNotFoundError: File database_konten2.json does not exist

In [242]:
class Recommendation():
    def __init__(self,dataframe,user_input):
        self.tfidfvec = TfidfVectorizer()
        self.dataframe = self.prepare_dataframe(dataframe)
        self.user_input = user_input
        
    def prepare_dataframe(self,dataframe):
        self.dataframe = pd.read_json(dataframe,orient='records')
        self.dataframe['cosim'] = 0
        self.dataframe['tags'] = self.dataframe['tags'].apply(lambda x: ' '.join(ast.literal_eval(x)))
        self.dataframe['text'] = self.dataframe['title']+' '+self.dataframe['summary']+' '+self.dataframe['content'] + ' ' + self.dataframe['tags']
        self.dataframe.drop(['title','summary','content','tags','imgUrl','link'],axis=1,inplace=True)
        self.dataframe['text'] = self.dataframe['text'].apply(lambda x: clean_text(x))
        return self.dataframe
    
    def clean_text(self,text):
        self.text = text.lower()
        self.text = self.text.strip()
        self.text = re.sub('[^a-z ]','',self.text)
        self.text = ' '.join([word for word in self.text.split(' ') if not word in list_stopwords])
        self.text = self.text.strip()
        return self.text
    
    def recomendations(self):
        self.vector = self.tfidfvec.fit_transform(self.dataframe['text']).toarray()
        self.user_input = self.clean_text(self.user_input)
        self.vector_user = self.tfidfvec.transform([self.user_input]).toarray()
        self.cos_sim = cosine_similarity(self.vector,self.vector_user)
        self.dataframe['cosim'] = self.cos_sim
        if np.max(self.dataframe['cosim'])<=0.001:
            return self.dataframe.sample(10)
        return self.dataframe.sort_values('cosim',ascending=False).loc[:10]

In [243]:
person1 = Recommendation('./dataset/database_konten_fixed.json','p')
person1.recomendations()

,id,author,createdAt,viewers,likes,cosim,text
16,62d0788c-0044-11ee-812e-bb155eb672e5,dr. Rizal Fadli,19 April 2022,0,0,0.0,manfaat yoga kesehatan tubuh pikiran simak man...
157,6d4fb147-0044-11ee-9756-bb155eb672e5,Redaksi Halodoc,16 Januari 2019,0,0,0.0,takut gemuk hindari camilan halodoc jakarta ...
124,6a818a36-0044-11ee-8046-bb155eb672e5,dr. Verury Verona Handayani,06 Desember 2019,0,0,0.0,kesepian marah alasannya halodoc jakarta ken...
189,6fbe7329-0044-11ee-89a9-bb155eb672e5,dr. Rizal Fadli,16 Oktober 2020,0,0,0.0,alasan kue redakan gangguan kecemasan halodoc...
177,6ecdcb60-0044-11ee-9743-bb155eb672e5,dr. Rizal Fadli,27 November 2019,0,0,0.0,makanan ampuh mengurangi cemas halodoc jakart...
170,6e5306d5-0044-11ee-8312-bb155eb672e5,dr. Fadhli Rizal Makarim,07 Oktober 2022,0,0,0.0,cemas takut membaca kdrt membaca berita negati...
141,6bcedbec-0044-11ee-bddd-bb155eb672e5,dr. Rizal Fadli,22 Juli 2021,0,0,0.0,anak marah pandemi penyebabnya cemas bosan se...
75,66e82917-0044-11ee-ad01-bb155eb672e5,dr. Rizal Fadli,04 November 2022,0,0,0.0,kaitan amygdala hijack kesehatan mental faktan...
184,6f5a298c-0044-11ee-8d6d-bb155eb672e5,Redaksi Halodoc,03 Maret 2019,0,0,0.0,depresi cemas memicu mimpi buruk halodoc jaka...
62,660b22f8-0044-11ee-a41c-bb155eb672e5,dr. Rizal Fadli,12 Desember 2022,0,0,0.0,manfaat pijat refleksi kesehatan tubuh mental ...


In [1]:
from recommendation import Recommendation

In [3]:
person2 = Recommendation('dataset/database_konten_fixed.json','saya ingin makan makanan enak')
hasil = person2.recomendations()

In [55]:
dict_keys = hasil.to_dict().keys()
dict_values = hasil.to_dict().values()

In [40]:
recommendations = list(person2.recomendations().itertuples(index=False, name=None))
recommendation_list = [{'key': rec[0], 'title': rec[1],} for i,rec in enumerate(recommendations)]
recommendation_list

[{'key': '644b6d34-0044-11ee-93a3-bb155eb672e5',
  'title': 'Ini Menu Makanan Sehat untuk Keluarga yang Bisa Dicoba'},
 {'key': '64215258-0044-11ee-b842-bb155eb672e5',
  'title': 'Catat, Ini 4 Makanan Diet Sehat untuk Menurunkan Berat Badan'},
 {'key': '63041159-0044-11ee-8c98-bb155eb672e5',
  'title': 'Ini Berbagai Makanan Sehat dan Bergizi untuk Lansia'},
 {'key': '63366c54-0044-11ee-a3b3-bb155eb672e5',
  'title': '8 Makanan Sehat untuk Menjaga Kesehatan Keluarga'},
 {'key': '63ad6bef-0044-11ee-a4a2-bb155eb672e5',
  'title': 'Tidak Perlu Mahal, Ini 5 Makanan Sehat untuk Ibu Hamil'},
 {'key': '638bb166-0044-11ee-8757-bb155eb672e5',
  'title': 'Ide Makanan Sehat untuk Makan Malam Keluarga'},
 {'key': '63de2139-0044-11ee-a403-bb155eb672e5',
  'title': 'Kaya Nutrisi, Ini 4 Pilihan Makanan Sehat untuk Lansia'},
 {'key': '647d0eaa-0044-11ee-9431-bb155eb672e5',
  'title': 'Pentingnya Konsumsi Makanan Sehat agar Kulit Glowing'},
 {'key': '6379ff55-0044-11ee-92fd-bb155eb672e5',
  'title': 'Ja

In [340]:
model_path = './wiki.id.case.model'
pretrained_model = Word2Vec.load(model_path)

In [249]:
# Step 2: Build the corpus
def build_corpus(df, text_column):
    corpus = []
    for text in df[text_column]:
        corpus.append(text)
    return corpus

# Step 3: Train the Word2Vec model
def train_word2vec(corpus, vector_size=100, window=5, min_count=1, epochs=5):
    model = Word2Vec(corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs)
    return model

In [ ]:
# Step 4: Compute user-feed similarity
def calculate_user_feed_similarity(user_embedding, feed_embeddings):
    return cosine_similarity(user_embedding.reshape(1, -1), list(feed_embeddings.values()))[0]

In [294]:
model_path = './wiki.id.case.model'
model = Word2Vec.load(model_path)
feed_embeddings = {feed_id: sum(model.wv[word] for word in text.split() if word in model.wv)
                   for feed_id, text in df[['id', 'text']].values}
# Step 5: Generate recommendations for a user based on user input text
user_input_text = 'saya ingin menjaga kesehatan mental saya lebih baik lagi'
user_embedding = sum(model.wv[word] for word in user_input_text.split() if word in model.wv)
user_feed_similarity = calculate_user_feed_similarity(user_embedding, feed_embeddings)

# Find most relevant feeds to the user based on similarity scores
relevant_feeds_indices = user_feed_similarity.argsort()[-3:][::-1]  # Get top 3 relevant feeds
recommended_feeds = [list(feed_embeddings.keys())[i] for i in relevant_feeds_indices]

df[df['id'].isin(recommended_feeds)]

,id,title,summary,imgUrl,content,author,createdAt,tags,link,viewers,likes,text,cosim
59,65d8f65b-0044-11ee-99e9-bb155eb672e5,"Penting, Ini 6 Cara Jaga Kesehatan Mental di A...",“Ada beberapa cara yang direkomendasikan untuk...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Kini tahun baru 2023 tingga...",dr. Rizal Fadli,30 Desember 2022,kesehatan mental,https://www.halodoc.com/artikel/penting-ini-6-...,0,0,jaga kesehatan mental direkomendasikan menjaga...,0.058684
65,663f4d56-0044-11ee-9f1f-bb155eb672e5,Ini Cara Menjaga Kesehatan Mental Anak yang Wa...,"""Orang tua punya peran yang besar untuk menjag...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Kesehatan mental anak sama ...",dr. Fadhli Rizal Makarim,25 November 2022,kesehatan mental anak,https://www.halodoc.com/artikel/ini-cara-menja...,0,0,menjaga kesehatan mental anak wajib ketahui or...,0.038865
68,66722970-0044-11ee-b63b-bb155eb672e5,Ini Cara Mudah Menjaga Kesehatan Mental Ibu Pe...,"“Setiap orang wajib menjaga kesehatan mental, ...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Ibu pekerja termasuk katego...",dr. Fadhli Rizal Makarim,25 November 2022,kesehatan mental wanita,https://www.halodoc.com/artikel/ini-cara-mudah...,0,0,mudah menjaga kesehatan mental pekerja orang w...,0.000000


In [211]:
person2 = Recommendation('dataset/database_konten_fixed.json',user_input_text)
df[df['id'].isin(person2.recomendations())][:3]

,id,title,summary,imgUrl,content,author,createdAt,tags,link,viewers,likes,text,cosim
0,61a420e4-0044-11ee-bbbb-bb155eb672e5,Tips Menjaga Kebugaran Lansia yang Perlu Diket...,"“Seiring dengan bertambahnya umur, menjaga keb...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seiring bertambahnya usia, ...",dr. Rizal Fadli,29 Mei 2023,kesehatan lansia kebugaran,https://www.halodoc.com/artikel/tips-menjaga-k...,0,0,tips menjaga kebugaran lansia seiring bertamba...,0.11204
1,61b5264f-0044-11ee-ac36-bb155eb672e5,"Jarang Diketahui, Ini 12 Manfaat Daun Seledri ...",“Ada berbagai manfaat seledri untuk kesehatan....,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seledri merupakan sayuran y...",dr. Fadhli Rizal Makarim,30 Mei 2023,kesehatan makanan,https://www.halodoc.com/artikel/jarang-diketah...,0,0,jarang manfaat daun seledri kesehatan manfaat...,0.01412
2,61c727ae-0044-11ee-bd96-bb155eb672e5,9 Obat Herbal yang Ampuh Atasi Sakit Lutut dan...,“Obat herbal dapat menjadi pilihan alternatif ...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Sakit lutut dan sendi adala...",dr. Rizal Fadli,10 Mei 2023,herbal kesehatan obat,https://www.halodoc.com/artikel/9-obat-herbal-...,0,0,obat herbal ampuh atasi sakit lutut sendi obat...,0.00000
